 <a name='ind'/>

## <ins>Indice</ins>:

#### 0- [Importaciones](#imp)
#### 1- [Exploración](#exp)
#### 2- [Limpieza](#clean)
***

En este notebook prepararemos el [dataset](../data/train.csv) que se nos ha proporcionado para poder aplicar un modelo de Machine Learning, que nos permita predecir el precio de un ordenador.

### Importaciones
***

In [158]:
import pandas as pd
import numpy as np
pd.options.plotting.backend = "plotly"
pd.set_option('display.max_columns', None) 

import re

import plotly.express as px

<a name='exp'/>

###### ⬆️ [Indice](#ind)

### Exploración
***

Vamos a revisar la estructura de los datos, y sus tipos y obtener unas primeras conclusiones para iniciar la limpieza

In [159]:
data_train = pd.read_csv('../data/raw/train.csv')
data_test = pd.read_csv('../data/raw/test.csv')

In [160]:
print(data_train.shape)
data_train.head()

(977, 13)


,Manufacturer,Model Name,Category,Screen Size,Screen,CPU,RAM,Storage,GPU,Operating System,Operating System Version,Weight,Price
0,Apple,MacBook Pro,Ultrabook,"13.3""",IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,NaN,1.37kg,11912523.48
1,Apple,Macbook Air,Ultrabook,"13.3""",1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,NaN,1.34kg,7993374.48
2,HP,250 G6,Notebook,"15.6""",Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,NaN,1.86kg,5112900.00
3,Apple,MacBook Pro,Ultrabook,"15.4""",IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,NaN,1.83kg,22563005.40
4,Apple,MacBook Pro,Ultrabook,"13.3""",IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,NaN,1.37kg,16037611.20


In [161]:
# Formateamos columnas

columns = list(map(lambda x:x.strip(),data_train.columns))
data_train.columns = list(map(lambda x:x.replace(' ','_'),columns))

In [162]:
columns = list(map(lambda x:x.strip(),data_test.columns))
data_test.columns = list(map(lambda x:x.replace(' ','_'),columns))

In [163]:
data_train.head()

,Manufacturer,Model_Name,Category,Screen_Size,Screen,CPU,RAM,Storage,GPU,Operating_System,Operating_System_Version,Weight,Price
0,Apple,MacBook Pro,Ultrabook,"13.3""",IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,NaN,1.37kg,11912523.48
1,Apple,Macbook Air,Ultrabook,"13.3""",1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,NaN,1.34kg,7993374.48
2,HP,250 G6,Notebook,"15.6""",Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,NaN,1.86kg,5112900.00
3,Apple,MacBook Pro,Ultrabook,"15.4""",IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,NaN,1.83kg,22563005.40
4,Apple,MacBook Pro,Ultrabook,"13.3""",IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,NaN,1.37kg,16037611.20


In [164]:
data_train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977 entries, 0 to 976
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Manufacturer              977 non-null    object 
 1   Model_Name                977 non-null    object 
 2   Category                  977 non-null    object 
 3   Screen_Size               977 non-null    object 
 4   Screen                    977 non-null    object 
 5   CPU                       977 non-null    object 
 6   RAM                       977 non-null    object 
 7   Storage                   977 non-null    object 
 8   GPU                       977 non-null    object 
 9   Operating_System          977 non-null    object 
 10  Operating_System_Version  841 non-null    object 
 11  Weight                    977 non-null    object 
 12  Price                     977 non-null    float64
dtypes: float64(1), object(12)
memory usage: 780.0 KB


In [165]:
data_train.describe().T

,count,mean,std,min,25%,50%,75%,max
Price,977.0,1.001899e+07,6.306430e+06,1706374.8,5326308.0,8527428.0,13115700.0,54232308.0


In [166]:
data_train.describe(include='object').T

,count,unique,top,freq
Manufacturer,977,19,Dell,232
Model_Name,977,488,XPS 13,22
Category,977,6,Notebook,549
Screen_Size,977,18,"15.6""",494
Screen,977,38,Full HD 1920x1080,397
CPU,977,106,Intel Core i5 7200U 2.5GHz,151
RAM,977,8,8GB,483
Storage,977,36,256GB SSD,318
GPU,977,98,Intel HD Graphics 620,225
Operating_System,977,7,Windows,837


### <ins>Primeras impresiones</ins>:

- Solo tenemos nulos en la columna versión
- Weight la pasaremos a float. Revisando que estén todos los pesos en las mismas unidades
- RAM y Storage, se pueden pasar a int, si todas están en GB o MB, si no hay que hacer la conversión adecuada
- Hay mucha varaición en el precio. ¿Outliers?
- En CPU podemos explosionarla en valores como marca, nucleos GHZ y versión
- Category admite también one-hot encoding, solo 6 categorías
- Model Name es más peleaguda, si la pasamos a one-hot encoding sería recomendable bajar dimensiones
- Manufacturer 19 valores uno no debe ser más importante que otro
- Sería interesante ver como se comporta el modelo si aplicamos una clusterización y luego un aplicamos un modelo de regresión

<a name='clean'/>

###### ⬆️ [Indice](#ind)

### Limpieza
***

Gestionaremos nulos y cambiaremos tipos de datos de cada columna

#### Columnas susceptibles de pasar a numérico

In [167]:
# Clumna Price. Presencia de outliers
px.box(data_train.Price)

In [168]:
(data_train.Price>24.67201*1e6).sum()

29

In [169]:
outliers_index = data_train[(data_train.Price>24.67201*1e6)].index

Tenemos 29 outliers, vamos serpararlos pero a la hora de entrenar el modelo probaremos el dataset con y sin outliers

A la hora de transformar y limpiar, vamos a concatenar los dos dataset de train y test para hacer las mismas transformaciones en ambos, una vez hecha la limpieza y la transformación, separaremos de nuevo los dos datasets para entrenar y evaluar los distintos modelos de machine learning

In [170]:
data = pd.concat([data_train.drop(columns='Price'),data_test],axis=0).reset_index(drop=True)
data.loc[data_train.shape[0]] == data_test.loc[0]    # posición de inicio en el dataframe de los datos de test

Manufacturer                True
Model_Name                  True
Category                    True
Screen_Size                 True
Screen                      True
CPU                         True
RAM                         True
Storage                     True
GPU                         True
Operating_System            True
Operating_System_Version    True
Weight                      True
dtype: bool

In [171]:
# Columna Wieight. Parace que todo está en Kg
data.Weight.unique()

array(['1.37kg', '1.34kg', '1.86kg', '1.83kg', '2.1kg', '2.04kg', '1.3kg',
       '1.6kg', '2.2kg', '0.92kg', '1.22kg', '0.98kg', '2.5kg', '1.62kg',
       '1.91kg', '2.3kg', '1.35kg', '1.88kg', '1.89kg', '1.65kg',
       '2.71kg', '1.2kg', '1.44kg', '2.8kg', '2kg', '2.65kg', '2.77kg',
       '3.2kg', '0.69kg', '1.49kg', '2.4kg', '2.13kg', '2.43kg', '1.7kg',
       '1.4kg', '1.8kg', '1.9kg', '3kg', '1.252kg', '2.7kg', '2.02kg',
       '1.63kg', '1.96kg', '1.21kg', '2.45kg', '1.25kg', '1.5kg',
       '2.62kg', '1.38kg', '1.58kg', '1.85kg', '1.23kg', '1.26kg',
       '2.16kg', '2.36kg', '2.05kg', '1.32kg', '1.75kg', '0.97kg',
       '2.9kg', '2.56kg', '1.48kg', '1.74kg', '1.1kg', '1.56kg', '2.03kg',
       '1.05kg', '4.4kg', '1.90kg', '1.29kg', '2.0kg', '1.95kg', '2.06kg',
       '1.12kg', '1.42kg', '3.49kg', '3.35kg', '2.23kg', '4.42kg',
       '2.69kg', '2.37kg', '4.7kg', '3.6kg', '2.08kg', '4.3kg', '1.68kg',
       '1.41kg', '4.14kg', '2.18kg', '2.24kg', '2.67kg', '2.14kg',
       '1.

In [172]:
# Hay un registro que no sus dos últimos caracteres no es Kg
data.Weight.apply(lambda x: x[-2:] != 'kg').sum()

1

In [173]:
data[data.Weight.apply(lambda x: x[-2:] != 'kg')]

,Manufacturer,Model_Name,Category,Screen_Size,Screen,CPU,RAM,Storage,GPU,Operating_System,Operating_System_Version,Weight
1060,Asus,Rog G752VL-GC088D,Gaming,"17.3""",IPS Panel Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16GB,1TB HDD,Nvidia GeForce GTX 965M,No OS,NaN,4kgs


In [174]:
data.loc[data.Weight.apply(lambda x: x[-2:] != 'kg'),'Weight'] = '4kg'

In [175]:
data.Weight = data.Weight.apply(lambda x: float(x[:-2]))

In [176]:
# Columna RAM
data.RAM.unique()

array(['8GB', '16GB', '4GB', '2GB', '12GB', '6GB', '32GB', '24GB', '64GB'],
      dtype=object)

In [177]:
data.RAM = data.RAM.apply(lambda x: np.int8(x[:-2]))

In [178]:
# Columna Storage

data[data.Storage.str.contains(r'+', regex=False)]['Storage'].unique()

array(['128GB SSD +  1TB HDD', '256GB SSD +  256GB SSD',
       '256GB SSD +  1TB HDD', '256GB SSD +  2TB HDD',
       '512GB SSD +  1TB HDD', '256GB SSD +  500GB HDD',
       '128GB SSD +  2TB HDD', '512GB SSD +  512GB SSD',
       '512GB SSD +  256GB SSD', '512GB SSD +  2TB HDD',
       '64GB Flash Storage +  1TB HDD', '1TB HDD +  1TB HDD',
       '1TB SSD +  1TB HDD', '512GB SSD +  1TB Hybrid',
       '256GB SSD +  1TB Hybrid'], dtype=object)

Esta columna tiene algunos valores que tenemos que tratar con cuidado. Podemos crear una columna con el valor total de almacenamiento, y otra con el tipo de almacenamiento. Hay que tener cuidado porque tenemos GB y TB

In [179]:
# Creamos función para obtener la parte numérica de la columna
def get_numeric(text):
    patron = r'\d+'                            # Patrón para encontrar una o más secuencias de dígitos
    matchs = re.findall(patron, text)
    numeros = [int(m) for m in matchs]         # Convertir las coincidencias a enteros
    return sum(numeros)

In [180]:
# Creamos función para obtener el dato en las mismas unidades
def get_storage(x):
    groups = x.split(' + ')
    storage= 0
    for group in groups:
        if 'GB' in group:
            storage += get_numeric(group)
        elif 'TB' in group:
            storage += get_numeric(group)*1024
    return storage

In [181]:
data['Storage_Capacity'] = data.Storage.apply(get_storage)

In [182]:
# Nos interesa obtener el tipo de disco duro de cada ordenador
def get_disk_type(x):
    groups = x.split(' + ')
    patron = r"(?<=B\s).+"
    tipo = []
    for group in groups:
        tipo.append(re.findall(patron,group)[0])
    return ' + '.join(tipo)

In [183]:
data['Storage_Type'] = data.Storage.apply(get_disk_type)

In [184]:
data.Storage_Type.unique()

array(['SSD', 'Flash Storage', 'HDD', 'SSD + HDD', 'SSD + SSD', 'Hybrid',
       'Flash Storage + HDD', 'HDD + HDD', 'SSD + Hybrid'], dtype=object)

In [185]:
# Unificamos tipos para evitar duplicidades

data.loc[data.Storage_Type == 'SSD + SSD','Storage_Type'] = 'SSD' 
data.loc[data.Storage_Type == 'HDD + HDD','Storage_Type'] = 'HDD'

In [186]:
data.Storage_Type.unique()

array(['SSD', 'Flash Storage', 'HDD', 'SSD + HDD', 'Hybrid',
       'Flash Storage + HDD', 'SSD + Hybrid'], dtype=object)

In [187]:
data.tail()

,Manufacturer,Model_Name,Category,Screen_Size,Screen,CPU,RAM,Storage,GPU,Operating_System,Operating_System_Version,Weight,Storage_Capacity,Storage_Type
1297,Lenovo,Yoga 500-14ISK,2 in 1 Convertible,"14.0""",IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6500U 2.5GHz,4,128GB SSD,Intel HD Graphics 520,Windows,10,1.80,128,SSD
1298,Lenovo,Yoga 900-13ISK,2 in 1 Convertible,"13.3""",IPS Panel Quad HD+ / Touchscreen 3200x1800,Intel Core i7 6500U 2.5GHz,16,512GB SSD,Intel HD Graphics 520,Windows,10,1.30,512,SSD
1299,Lenovo,IdeaPad 100S-14IBR,Notebook,"14.0""",1366x768,Intel Celeron Dual Core N3050 1.6GHz,2,64GB Flash Storage,Intel HD Graphics,Windows,10,1.50,64,Flash Storage
1300,HP,15-AC110nv (i7-6500U/6GB/1TB/Radeon,Notebook,"15.6""",1366x768,Intel Core i7 6500U 2.5GHz,6,1TB HDD,AMD Radeon R5 M330,Windows,10,2.19,1024,HDD
1301,Asus,X553SA-XX031T (N3050/4GB/500GB/W10),Notebook,"15.6""",1366x768,Intel Celeron Dual Core N3050 1.6GHz,4,500GB HDD,Intel HD Graphics,Windows,10,2.20,500,HDD


In [188]:
# Columna CPU
data.CPU.unique()

array(['Intel Core i5 2.3GHz', 'Intel Core i5 1.8GHz',
       'Intel Core i5 7200U 2.5GHz', 'Intel Core i7 2.7GHz',
       'Intel Core i5 3.1GHz', 'AMD A9-Series 9420 3GHz',
       'Intel Core i7 2.2GHz', 'Intel Core i7 8550U 1.8GHz',
       'Intel Core i5 8250U 1.6GHz', 'Intel Core i3 6006U 2GHz',
       'Intel Core i7 2.8GHz', 'Intel Core M m3 1.2GHz',
       'Intel Core i7 7500U 2.7GHz', 'Intel Core i7 2.9GHz',
       'Intel Core i3 7100U 2.4GHz', 'Intel Atom x5-Z8350 1.44GHz',
       'Intel Core i5 7300HQ 2.5GHz', 'AMD E-Series E2-9000e 1.5GHz',
       'Intel Core i5 1.6GHz', 'Intel Core i7 8650U 1.9GHz',
       'Intel Atom x5-Z8300 1.44GHz', 'AMD E-Series E2-6110 1.5GHz',
       'AMD A6-Series 9220 2.5GHz',
       'Intel Celeron Dual Core N3350 1.1GHz',
       'Intel Core i3 7130U 2.7GHz', 'Intel Core i7 7700HQ 2.8GHz',
       'Intel Core i5 2.0GHz', 'AMD Ryzen 1700 3GHz',
       'Intel Pentium Quad Core N4200 1.1GHz',
       'Intel Atom x5-Z8550 1.44GHz',
       'Intel Celeron Du

In [189]:
# Vemos que la mayoría son intel. Comprobamos los que no son intel
data[~data.CPU.str.contains('Intel')]['CPU'].unique()

array(['AMD A9-Series 9420 3GHz', 'AMD E-Series E2-9000e 1.5GHz',
       'AMD E-Series E2-6110 1.5GHz', 'AMD A6-Series 9220 2.5GHz',
       'AMD Ryzen 1700 3GHz', 'AMD FX 9830P 3GHz',
       'AMD E-Series 6110 1.5GHz', 'AMD A6-Series 9220 2.9GHz',
       'AMD E-Series 9000e 1.5GHz', 'AMD A10-Series A10-9620P 2.5GHz',
       'AMD A6-Series A6-9220 2.5GHz', 'AMD A10-Series 9600P 2.4GHz',
       'AMD A8-Series 7410 2.2GHz', 'AMD A12-Series 9720P 2.7GHz',
       'AMD A12-Series 9720P 3.6GHz', 'AMD Ryzen 1600 3.2GHz',
       'AMD A10-Series 9620P 2.5GHz', 'AMD E-Series 7110 1.8GHz',
       'AMD A9-Series A9-9420 3GHz', 'AMD E-Series E2-9000 2.2GHz',
       'AMD A9-Series 9420 2.9GHz', 'AMD A6-Series 7310 2GHz',
       'AMD A12-Series 9700P 2.5GHz', 'AMD A4-Series 7210 2.2GHz',
       'AMD FX 8800P 2.1GHz', 'Samsung Cortex A72&A53 2.0GHz',
       'AMD E-Series 9000 2.2GHz', 'AMD A9-Series 9410 2.9GHz'],
      dtype=object)

Tenemos dos fabricantes de procesadores: AMD e Intel

In [190]:
data['CPU_brand'] = data.CPU.apply(lambda x: x.split()[0])

In [191]:
# Función que devuelve la frecuencia de la CPU

def get_GHz(x):
    patron = r"(\d+(?:\.\d+)?)GHz"
    return float(re.findall(patron,x)[0])

In [192]:
data['CPU_freq'] = data.CPU.apply(get_GHz)

Sería interesante tener el número de nucleos. Nos ayudamos de chat GPT para que nos de esta información y obtenemos un diccionario con todos los nucleos de cada procesador

In [193]:
cpu_cores = {
'Intel Core i5 2.3GHz': 4,
'Intel Core i5 1.8GHz': 4,
'Intel Core i5 7200U 2.5GHz': 2,
'Intel Core i7 2.7GHz': 4,
'Intel Core i5 3.1GHz': 4,
'AMD A9-Series 9420 3GHz': 2,
'Intel Core i7 2.2GHz': 4,
'Intel Core i7 8550U 1.8GHz': 4,
'Intel Core i5 8250U 1.6GHz': 4,
'Intel Core i3 6006U 2GHz': 2,
'Intel Core i7 2.8GHz': 4,
'Intel Core M m3 1.2GHz': 2,
'Intel Core i7 7500U 2.7GHz': 2,
'Intel Core i7 2.9GHz': 4,
'Intel Core i3 7100U 2.4GHz': 2,
'Intel Atom x5-Z8350 1.44GHz': 4,
'Intel Core i5 7300HQ 2.5GHz': 4,
'AMD E-Series E2-9000e 1.5GHz': 2,
'Intel Core i5 1.6GHz': 4,
'Intel Core i7 8650U 1.9GHz': 4,
'Intel Atom x5-Z8300 1.44GHz': 4,
'AMD E-Series E2-6110 1.5GHz': 4,
'AMD A6-Series 9220 2.5GHz': 2,
'Intel Celeron Dual Core N3350 1.1GHz': 2,
'Intel Core i3 7130U 2.7GHz': 2,
'Intel Core i7 7700HQ 2.8GHz': 4,
'Intel Core i5 2.0GHz': 4,
'AMD Ryzen 1700 3GHz': 8,
'Intel Pentium Quad Core N4200 1.1GHz': 4,
'Intel Atom x5-Z8550 1.44GHz': 4,
'Intel Celeron Dual Core N3060 1.6GHz': 2,
'Intel Core i5 1.3GHz': 4,
'AMD FX 9830P 3GHz': 4,
'Intel Core i7 7560U 2.4GHz': 2,
'AMD E-Series 6110 1.5GHz': 4,
'Intel Core i5 6200U 2.3GHz': 2,
'Intel Core M 6Y75 1.2GHz': 2,
'Intel Core i5 7500U 2.7GHz': 2,
'Intel Core i3 6006U 2.2GHz': 2,
'AMD A6-Series 9220 2.9GHz': 2,
'Intel Core i7 6920HQ 2.9GHz': 4,
'Intel Core i5 7Y54 1.2GHz': 2,
'Intel Core i7 7820HK 2.9GHz': 4,
'Intel Xeon E3-1505M V6 3GHz': 4,
'Intel Core i7 6500U 2.5GHz': 2,
'AMD E-Series 9000e 1.5GHz': 2,
'AMD A10-Series A10-9620P 2.5GHz': 4,
'AMD A6-Series A6-9220 2.5GHz': 2,
'Intel Core i5 2.9GHz': 4,
'Intel Core i7 6600U 2.6GHz': 2,
'Intel Core i3 6006U 2.0GHz': 2,
'Intel Celeron Dual Core 3205U 1.5GHz': 2,
'Intel Core i7 7820HQ 2.9GHz': 4,
'AMD A10-Series 9600P 2.4GHz': 4,
'Intel Core i7 7600U 2.8GHz': 2,
'AMD A8-Series 7410 2.2GHz': 4,
'Intel Celeron Dual Core 3855U 1.6GHz': 2,
'Intel Pentium Quad Core N3710 1.6GHz': 4,
'AMD A12-Series 9720P 2.7GHz': 4,
'Intel Core i5 7300U 2.6GHz': 2,
'AMD A12-Series 9720P 3.6GHz': 4,
'Intel Celeron Quad Core N3450 1.1GHz': 4,
'Intel Celeron Dual Core N3060 1.60GHz': 2,
'Intel Core i5 6440HQ 2.6GHz': 4,
'Intel Core i7 6820HQ 2.7GHz': 4,
'AMD Ryzen 1600 3.2GHz': 6,
'Intel Core i7 7Y75 1.3GHz': 2,
'Intel Core i5 7440HQ 2.8GHz': 4,
'Intel Core i7 7660U 2.5GHz': 2,
'Intel Core i7 7700HQ 2.7GHz': 4,
'Intel Core M m3-7Y30 2.2GHz': 2,
'Intel Core i5 7Y57 1.2GHz': 2,
'Intel Core i7 6700HQ 2.6GHz': 4,
'Intel Core i3 6100U 2.3GHz': 2,
'AMD A10-Series 9620P 2.5GHz': 4,
'AMD E-Series 7110 1.8GHz': 4,
'Intel Celeron Dual Core N3350 2.0GHz': 2,
'AMD A9-Series A9-9420 3GHz': 2,
'Intel Core i7 6820HK 2.7GHz': 4,
'Intel Core M 7Y30 1.0GHz': 2,
'Intel Xeon E3-1535M v6 3.1GHz': 4,
'Intel Celeron Quad Core N3160 1.6GHz': 4,
'Intel Core i5 6300U 2.4GHz': 2,
'Intel Core i3 6100U 2.1GHz': 2,
'AMD E-Series E2-9000 2.2GHz': 2,
'Intel Celeron Dual Core N3050 1.6GHz': 2,
'Intel Core M M3-6Y30 0.9GHz': 2,
'AMD A9-Series 9420 2.9GHz': 2,
'Intel Core i5 6300HQ 2.3GHz': 4,
'AMD A6-Series 7310 2GHz': 4,
'Intel Atom Z8350 1.92GHz': 4,
'Intel Xeon E3-1535M v5 2.9GHz': 4,
'Intel Core i5 6260U 1.8GHz': 2,
'Intel Pentium Dual Core N4200 1.1GHz': 4,
'Intel Celeron Quad Core N3710 1.6GHz': 4,
'Intel Core M 1.2GHz': 2,
'AMD A12-Series 9700P 2.5GHz': 4,
'Intel Core i7 7500U 2.5GHz': 2,
'Intel Pentium Dual Core 4405U 2.1GHz': 2,
'AMD A4-Series 7210 2.2GHz': 2,
'Intel Core i7 6560U 2.2GHz': 2,
'Intel Core M m7-6Y75 1.2GHz': 2,
'AMD FX 8800P 2.1GHz': 4,
'Intel Core M M7-6Y75 1.2GHz': 2,
'Intel Core i5 7200U 2.50GHz': 2,
'Intel Core i5 7200U 2.70GHz': 2,
'Intel Atom X5-Z8350 1.44GHz': 4,
'Intel Core i5 7200U 2.7GHz': 2,
'Intel Core M 1.1GHz': 1,
'Intel Pentium Dual Core 4405Y 1.5GHz': 2,
'Intel Pentium Quad Core N3700 1.6GHz': 4,
'Intel Core M 6Y54 1.1GHz': 2,
'Intel Core i7 6500U 2.50GHz': 2,
'Intel Celeron Dual Core N3350 2GHz': 2,
'Samsung Cortex A72&A53 2.0GHz': 8,
'AMD E-Series 9000 2.2GHz': 2,
'Intel Core M 6Y30 0.9GHz': 2,
'AMD A9-Series 9410 2.9GHz': 2
}

In [194]:
data['CPU_cores'] = data.CPU.apply(lambda x: cpu_cores[x])

In [195]:
data.head()

,Manufacturer,Model_Name,Category,Screen_Size,Screen,CPU,RAM,Storage,GPU,Operating_System,Operating_System_Version,Weight,Storage_Capacity,Storage_Type,CPU_brand,CPU_freq,CPU_cores
0,Apple,MacBook Pro,Ultrabook,"13.3""",IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,NaN,1.37,128,SSD,Intel,2.3,4
1,Apple,Macbook Air,Ultrabook,"13.3""",1440x900,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,NaN,1.34,128,Flash Storage,Intel,1.8,4
2,HP,250 G6,Notebook,"15.6""",Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,No OS,NaN,1.86,256,SSD,Intel,2.5,2
3,Apple,MacBook Pro,Ultrabook,"15.4""",IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,512GB SSD,AMD Radeon Pro 455,macOS,NaN,1.83,512,SSD,Intel,2.7,4
4,Apple,MacBook Pro,Ultrabook,"13.3""",IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,NaN,1.37,256,SSD,Intel,3.1,4


In [196]:
def get_cpu_model(x):
    patron = r"(?!(?:AMD|Intel|\d+(?:\.\d+)?GHz)\b)\b\w+\b"
    modelo = re.findall(patron,x)
    return ' '.join(modelo)

In [197]:
data['CPU_model'] = data.CPU.apply(get_cpu_model)

In [198]:
data.head()

,Manufacturer,Model_Name,Category,Screen_Size,Screen,CPU,RAM,Storage,GPU,Operating_System,Operating_System_Version,Weight,Storage_Capacity,Storage_Type,CPU_brand,CPU_freq,CPU_cores,CPU_model
0,Apple,MacBook Pro,Ultrabook,"13.3""",IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,NaN,1.37,128,SSD,Intel,2.3,4,Core i5
1,Apple,Macbook Air,Ultrabook,"13.3""",1440x900,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,NaN,1.34,128,Flash Storage,Intel,1.8,4,Core i5
2,HP,250 G6,Notebook,"15.6""",Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,No OS,NaN,1.86,256,SSD,Intel,2.5,2,Core i5 7200U
3,Apple,MacBook Pro,Ultrabook,"15.4""",IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,512GB SSD,AMD Radeon Pro 455,macOS,NaN,1.83,512,SSD,Intel,2.7,4,Core i7
4,Apple,MacBook Pro,Ultrabook,"13.3""",IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,NaN,1.37,256,SSD,Intel,3.1,4,Core i5


In [199]:
# Columna Screen_Size

data.Screen_Size.unique()

array(['13.3"', '15.6"', '15.4"', '14.0"', '12.0"', '11.6"', '17.3"',
       '10.1"', '13.5"', '12.5"', '13.0"', '18.4"', '13.9"', '12.3"',
       '17.0"', '15.0"', '14.1"', '11.3"'], dtype=object)

In [200]:
data.Screen_Size = data.Screen_Size.apply(lambda x:float(x[:-1]))

In [201]:
# Columna Screen
data.Screen.unique()

array(['IPS Panel Retina Display 2560x1600', '1440x900',
       'Full HD 1920x1080', 'IPS Panel Retina Display 2880x1800',
       '1366x768', 'IPS Panel Full HD 1920x1080',
       'IPS Panel Retina Display 2304x1440',
       'IPS Panel Full HD / Touchscreen 1920x1080',
       'Full HD / Touchscreen 1920x1080',
       'Touchscreen / Quad HD+ 3200x1800',
       'IPS Panel Touchscreen 1920x1200', 'Touchscreen 2256x1504',
       'Quad HD+ / Touchscreen 3200x1800', 'IPS Panel 1366x768',
       'IPS Panel 4K Ultra HD / Touchscreen 3840x2160',
       'IPS Panel Full HD 2160x1440',
       '4K Ultra HD / Touchscreen 3840x2160', 'Touchscreen 2560x1440',
       '1600x900', 'IPS Panel 4K Ultra HD 3840x2160',
       '4K Ultra HD 3840x2160', 'Touchscreen 1366x768',
       'IPS Panel Full HD 1366x768', 'IPS Panel 2560x1440',
       'IPS Panel Full HD 2560x1440',
       'IPS Panel Retina Display 2736x1824', 'Touchscreen 2400x1600',
       '2560x1440', 'IPS Panel Quad HD+ 2560x1440',
       'IPS Panel 

Separaremos tamaño de pantalla y tipo

In [202]:
def get_resolution(x):
    words = x.split()
    for word in words:
        if 'x' in word:
            return word
    return x

In [203]:
data.Screen.apply(get_resolution).unique()

array(['2560x1600', '1440x900', '1920x1080', '2880x1800', '1366x768',
       '2304x1440', '3200x1800', '1920x1200', '2256x1504', '3840x2160',
       '2160x1440', '2560x1440', '1600x900', '2736x1824', '2400x1600'],
      dtype=object)

In [204]:
data['Screen_Resolution'] = data.Screen.apply(get_resolution)

In [205]:
data.Screen.apply(lambda x: ' '.join(x.split()[:-1])).unique()   

array(['IPS Panel Retina Display', '', 'Full HD', 'IPS Panel Full HD',
       'IPS Panel Full HD / Touchscreen', 'Full HD / Touchscreen',
       'Touchscreen / Quad HD+', 'IPS Panel Touchscreen', 'Touchscreen',
       'Quad HD+ / Touchscreen', 'IPS Panel',
       'IPS Panel 4K Ultra HD / Touchscreen', '4K Ultra HD / Touchscreen',
       'IPS Panel 4K Ultra HD', '4K Ultra HD', 'IPS Panel Quad HD+',
       'IPS Panel Quad HD+ / Touchscreen',
       'IPS Panel Touchscreen / 4K Ultra HD', 'Touchscreen / Full HD',
       'Quad HD+', 'Touchscreen / 4K Ultra HD'], dtype=object)

In [206]:
data['Screen_Model'] = data.Screen.apply(lambda x: ' '.join(x.split()[:-1]))

Hay ordenadores que no tienen especificados el modelo de pantalla. Los catalogaremos como 'Unknown'

In [207]:
data.loc[data.Screen_Model=='']

,Manufacturer,Model_Name,Category,Screen_Size,Screen,CPU,RAM,Storage,GPU,Operating_System,Operating_System_Version,Weight,Storage_Capacity,Storage_Type,CPU_brand,CPU_freq,CPU_cores,CPU_model,Screen_Resolution,Screen_Model
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,NaN,1.34,128,Flash Storage,Intel,1.80,4,Core i5,1440x900,
5,Acer,Aspire 3,Notebook,15.6,1366x768,AMD A9-Series 9420 3GHz,4,500GB HDD,AMD Radeon R5,Windows,10,2.10,500,HDD,AMD,3.00,2,A9 Series 9420,1366x768,
7,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,256GB Flash Storage,Intel HD Graphics 6000,macOS,NaN,1.34,256,Flash Storage,Intel,1.80,4,Core i5,1440x900,
10,HP,250 G6,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,4,500GB HDD,Intel HD Graphics 620,No OS,NaN,1.86,500,HDD,Intel,2.50,2,Core i5 7200U,1366x768,
20,Asus,Vivobook E200HA,Netbook,11.6,1366x768,Intel Atom x5-Z8350 1.44GHz,2,32GB Flash Storage,Intel HD Graphics 400,Windows,10,0.98,32,Flash Storage,Intel,1.44,4,Atom x5 Z8350,1366x768,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295,HP,Stream 11-Y000na,Netbook,11.6,1366x768,Intel Celeron Dual Core N3060 1.6GHz,2,32GB Flash Storage,Intel HD Graphics 400,Windows,10,1.17,32,Flash Storage,Intel,1.60,2,Celeron Dual Core N3060,1366x768,
1296,Asus,X556UJ-XO044T (i7-6500U/4GB/500GB/GeForce,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,4,500GB HDD,Nvidia GeForce 920M,Windows,10,2.20,500,HDD,Intel,2.50,2,Core i7 6500U,1366x768,
1299,Lenovo,IdeaPad 100S-14IBR,Notebook,14.0,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2,64GB Flash Storage,Intel HD Graphics,Windows,10,1.50,64,Flash Storage,Intel,1.60,2,Celeron Dual Core N3050,1366x768,
1300,HP,15-AC110nv (i7-6500U/6GB/1TB/Radeon,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,6,1TB HDD,AMD Radeon R5 M330,Windows,10,2.19,1024,HDD,Intel,2.50,2,Core i7 6500U,1366x768,


In [208]:
data.loc[data.Screen_Model=='','Screen_Model'] = 'Unknown'

In [209]:
data.Screen_Model.value_counts()

Screen_Model
Full HD                                507
Unknown                                314
IPS Panel Full HD                      235
IPS Panel Full HD / Touchscreen         53
Full HD / Touchscreen                   47
Touchscreen                             32
IPS Panel Retina Display                17
Quad HD+ / Touchscreen                  15
IPS Panel Touchscreen                   13
IPS Panel 4K Ultra HD                   12
IPS Panel 4K Ultra HD / Touchscreen     11
IPS Panel                               11
4K Ultra HD / Touchscreen                9
4K Ultra HD                              7
IPS Panel Quad HD+ / Touchscreen         6
IPS Panel Quad HD+                       5
Quad HD+                                 3
IPS Panel Touchscreen / 4K Ultra HD      2
Touchscreen / Quad HD+                   1
Touchscreen / Full HD                    1
Touchscreen / 4K Ultra HD                1
Name: count, dtype: int64

In [210]:
data.head()

,Manufacturer,Model_Name,Category,Screen_Size,Screen,CPU,RAM,Storage,GPU,Operating_System,Operating_System_Version,Weight,Storage_Capacity,Storage_Type,CPU_brand,CPU_freq,CPU_cores,CPU_model,Screen_Resolution,Screen_Model
0,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,NaN,1.37,128,SSD,Intel,2.3,4,Core i5,2560x1600,IPS Panel Retina Display
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,NaN,1.34,128,Flash Storage,Intel,1.8,4,Core i5,1440x900,Unknown
2,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,No OS,NaN,1.86,256,SSD,Intel,2.5,2,Core i5 7200U,1920x1080,Full HD
3,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,512GB SSD,AMD Radeon Pro 455,macOS,NaN,1.83,512,SSD,Intel,2.7,4,Core i7,2880x1800,IPS Panel Retina Display
4,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,NaN,1.37,256,SSD,Intel,3.1,4,Core i5,2560x1600,IPS Panel Retina Display


#### Columnas categóricas

In [211]:
# Columna GPU
data.GPU.unique()

array(['Intel Iris Plus Graphics 640', 'Intel HD Graphics 6000',
       'Intel HD Graphics 620', 'AMD Radeon Pro 455',
       'Intel Iris Plus Graphics 650', 'AMD Radeon R5',
       'Intel Iris Pro Graphics', 'Nvidia GeForce MX150',
       'Intel UHD Graphics 620', 'Intel HD Graphics 520',
       'AMD Radeon Pro 555', 'AMD Radeon R5 M430',
       'Intel HD Graphics 615', 'AMD Radeon Pro 560',
       'Nvidia GeForce 940MX', 'Intel HD Graphics 400',
       'Nvidia GeForce GTX 1050', 'AMD Radeon R2', 'AMD Radeon 530',
       'Nvidia GeForce 930MX', 'Intel HD Graphics',
       'Intel HD Graphics 500', 'Nvidia GeForce 930MX ',
       'Nvidia GeForce GTX 1060', 'Nvidia GeForce 150MX',
       'Intel Iris Graphics 540', 'AMD Radeon RX 580',
       'Nvidia GeForce 920MX', 'AMD Radeon R4 Graphics', 'AMD Radeon 520',
       'Nvidia GeForce GTX 1070', 'Nvidia GeForce GTX 1050 Ti',
       'Nvidia GeForce MX130', 'AMD R4 Graphics',
       'Nvidia GeForce GTX 940MX', 'AMD Radeon RX 560',
       'Nvid

In [212]:
# Corrijo manualmente esta tarjeta gráfica
data[data.GPU=='Nvidia GeForce GTX 960<U+039C>']

,Manufacturer,Model_Name,Category,Screen_Size,Screen,CPU,RAM,Storage,GPU,Operating_System,Operating_System_Version,Weight,Storage_Capacity,Storage_Type,CPU_brand,CPU_freq,CPU_cores,CPU_model,Screen_Resolution,Screen_Model
611,Dell,Inspiron 7559,Gaming,15.6,Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16,1TB HDD,Nvidia GeForce GTX 960<U+039C>,Windows,10,2.59,1024,HDD,Intel,2.6,4,Core i7 6700HQ,1920x1080,Full HD
1217,Lenovo,IdeaPad Y700-15ISK,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,8,128GB SSD + 1TB HDD,Nvidia GeForce GTX 960<U+039C>,Windows,10,2.60,1152,SSD + HDD,Intel,2.6,4,Core i7 6700HQ,1920x1080,IPS Panel Full HD


In [213]:
data.loc[data.GPU=='Nvidia GeForce GTX 960<U+039C>','GPU'] = 'Nvidia GeForce GTX 960'

In [214]:
# Separamos fabricante 
data.GPU.apply(lambda x: x.split()[0]).unique()

array(['Intel', 'AMD', 'Nvidia', 'ARM'], dtype=object)

In [215]:
data['GPU_brand'] = data.GPU.apply(lambda x: x.split()[0])

In [216]:
# Columna Manufacturer. Parece que todo está correcto
data.Manufacturer.unique()

array(['Apple', 'HP', 'Acer', 'Asus', 'Dell', 'Lenovo', 'Chuwi', 'MSI',
       'Microsoft', 'Toshiba', 'Huawei', 'Xiaomi', 'Vero', 'Razer',
       'Mediacom', 'Samsung', 'Google', 'Fujitsu', 'LG'], dtype=object)

In [217]:
# Columna Model_name
data.Model_Name.unique()

array(['MacBook Pro', 'Macbook Air', '250 G6', 'Aspire 3',
       'ZenBook UX430UN', 'Swift 3', 'Inspiron 3567', 'MacBook 12"',
       'IdeaPad 320-15IKB', 'XPS 13', 'Vivobook E200HA',
       'Legion Y520-15IKBN', '255 G6', 'Inspiron 5379',
       '15-BS101nv (i7-8550U/8GB/256GB/FHD/W10)', 'MacBook Air',
       'Inspiron 5570', 'Latitude 5590', 'ProBook 470', 'LapBook 15.6"',
       'E402WA-GA010T (E2-6110/2GB/32GB/W10)',
       '17-ak001nv (A6-9220/4GB/500GB/Radeon', 'IdeaPad 120S-14IAP',
       'Inspiron 5770', 'ProBook 450',
       'X540UA-DM186 (i3-6006U/4GB/1TB/FHD/Linux)', 'Inspiron 7577',
       'X542UQ-GO005 (i5-7200U/8GB/1TB/GeForce', 'Aspire A515-51G',
       'Inspiron 7773', 'IdeaPad 320-15ISK', 'Rog Strix',
       'X751NV-TY001T (N4200/4GB/1TB/GeForce', 'Yoga Book', 'ProBook 430',
       'Inspiron 3576', '15-bs002nv (i3-6006U/4GB/128GB/FHD/W10)',
       'VivoBook Max', 'GS73VR 7RG',
       'X541UA-DM1897 (i3-6006U/4GB/256GB/FHD/Linux)', 'Vostro 5471',
       'IdeaPad 520S-1

El modelo del ordenar recoge en muchos casos información ya guardada en otras columnas. Dada su elevado número de valores únicos, hace que en principio no incluyamos esta columna para el entrenamiento del modelo

In [218]:
# Columna Category  y Operating System parecen que también está en orden
data.Category.unique()

array(['Ultrabook', 'Notebook', 'Netbook', 'Gaming', '2 in 1 Convertible',
       'Workstation'], dtype=object)

In [219]:
data.Category.value_counts()

Category
Notebook              727
Gaming                205
Ultrabook             196
2 in 1 Convertible    120
Workstation            29
Netbook                25
Name: count, dtype: int64

In [220]:
data.Operating_System.unique()

array(['macOS', 'No OS', 'Windows', 'Mac OS', 'Linux', 'Android',
       'Chrome OS'], dtype=object)

In [221]:
data.loc[data.Operating_System == 'macOS','Operating_System'] = 'Mac OS'
data.Operating_System.unique()

array(['Mac OS', 'No OS', 'Windows', 'Linux', 'Android', 'Chrome OS'],
      dtype=object)

In [222]:
data.Operating_System_Version.unique()

array([nan, '10', 'X', '10 S', '7'], dtype=object)

In [223]:
# Revisamos si todos los nulos están en los ordenadores que no tienen O

(data.Operating_System =='No OS').sum() == data.Operating_System_Version.isna().sum()

False

In [224]:
data[data.Operating_System_Version.isna()]['Operating_System'].value_counts()

Operating_System
No OS        66
Linux        62
Chrome OS    27
Mac OS       13
Android       2
Name: count, dtype: int64

Catalogaremos como 'No OS' aquellas versiones que no tengan sistema operativo instalado, y el resto como 'Unknown'

In [225]:
data.loc[(data.Operating_System_Version.isna())&(data.Operating_System=='No OS'),'Operating_System_Version'] = 'No OS'

In [226]:
data[data.Operating_System_Version.isna()]['Operating_System'].value_counts()

Operating_System
Linux        62
Chrome OS    27
Mac OS       13
Android       2
Name: count, dtype: int64

Todos los sitemas operativos Linux tienen como versión Unknown. Representan más de la mitad de los valores Unknown del dataset. Los SO de Linux son de código abierto y gratuitos, que haya uno u otra versión del mismo en un principio no debería afectar al precio, por lo que rellenaremos todos estos valores con la etiqueta 'Ubuntu' que es el SO más distribuido de LINUX

In [227]:
data.loc[data.Operating_System=='Linux','Operating_System_Version'] = 'Ubuntu'

In [228]:
data[data.Operating_System_Version.isna()]['Operating_System'].value_counts()

Operating_System
Chrome OS    27
Mac OS       13
Android       2
Name: count, dtype: int64

In [229]:
data.loc[data.Operating_System=='Chrome OS','Operating_System_Version'] = 'Chrome OS'

In [230]:
data[data.Operating_System_Version.isna()]['Operating_System'].value_counts()

Operating_System
Mac OS     13
Android     2
Name: count, dtype: int64

In [231]:
data.loc[data.Operating_System=='Android','Operating_System_Version'] = 'Android'

In [232]:
data[data.Operating_System=='Mac OS']['Operating_System_Version'].value_counts()

Operating_System_Version
X    8
Name: count, dtype: int64

In [233]:
data.loc[data.Operating_System=='Mac OS','Operating_System_Version'] = 'X'

In [234]:
data[data.Operating_System_Version.isna()]['Operating_System'].value_counts()

Series([], Name: count, dtype: int64)

In [235]:
data.head()

,Manufacturer,Model_Name,Category,Screen_Size,Screen,CPU,RAM,Storage,GPU,Operating_System,Operating_System_Version,Weight,Storage_Capacity,Storage_Type,CPU_brand,CPU_freq,CPU_cores,CPU_model,Screen_Resolution,Screen_Model,GPU_brand
0,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,128GB SSD,Intel Iris Plus Graphics 640,Mac OS,X,1.37,128,SSD,Intel,2.3,4,Core i5,2560x1600,IPS Panel Retina Display,Intel
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,Mac OS,X,1.34,128,Flash Storage,Intel,1.8,4,Core i5,1440x900,Unknown,Intel
2,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,No OS,No OS,1.86,256,SSD,Intel,2.5,2,Core i5 7200U,1920x1080,Full HD,Intel
3,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,512GB SSD,AMD Radeon Pro 455,Mac OS,X,1.83,512,SSD,Intel,2.7,4,Core i7,2880x1800,IPS Panel Retina Display,AMD
4,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,256GB SSD,Intel Iris Plus Graphics 650,Mac OS,X,1.37,256,SSD,Intel,3.1,4,Core i5,2560x1600,IPS Panel Retina Display,Intel


In [236]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1302 entries, 0 to 1301
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Manufacturer              1302 non-null   object 
 1   Model_Name                1302 non-null   object 
 2   Category                  1302 non-null   object 
 3   Screen_Size               1302 non-null   float64
 4   Screen                    1302 non-null   object 
 5   CPU                       1302 non-null   object 
 6   RAM                       1302 non-null   int8   
 7   Storage                   1302 non-null   object 
 8   GPU                       1302 non-null   object 
 9   Operating_System          1302 non-null   object 
 10  Operating_System_Version  1302 non-null   object 
 11  Weight                    1302 non-null   float64
 12  Storage_Capacity          1302 non-null   int64  
 13  Storage_Type              1302 non-null   object 
 14  CPU_bran

Tenemos el dataset limpio. Definiremos ahora las variables que tenemos que transformar. 

In [237]:
X = data.drop(columns=['Model_Name','Category','Screen','CPU','Storage'])
X.head()

,Manufacturer,Screen_Size,RAM,GPU,Operating_System,Operating_System_Version,Weight,Storage_Capacity,Storage_Type,CPU_brand,CPU_freq,CPU_cores,CPU_model,Screen_Resolution,Screen_Model,GPU_brand
0,Apple,13.3,8,Intel Iris Plus Graphics 640,Mac OS,X,1.37,128,SSD,Intel,2.3,4,Core i5,2560x1600,IPS Panel Retina Display,Intel
1,Apple,13.3,8,Intel HD Graphics 6000,Mac OS,X,1.34,128,Flash Storage,Intel,1.8,4,Core i5,1440x900,Unknown,Intel
2,HP,15.6,8,Intel HD Graphics 620,No OS,No OS,1.86,256,SSD,Intel,2.5,2,Core i5 7200U,1920x1080,Full HD,Intel
3,Apple,15.4,16,AMD Radeon Pro 455,Mac OS,X,1.83,512,SSD,Intel,2.7,4,Core i7,2880x1800,IPS Panel Retina Display,AMD
4,Apple,13.3,8,Intel Iris Plus Graphics 650,Mac OS,X,1.37,256,SSD,Intel,3.1,4,Core i5,2560x1600,IPS Panel Retina Display,Intel


Separamos de nuevo los datos en train y test, y los guardamos

In [238]:
X_train = X.loc[:data_train.shape[0]-1]

In [239]:
X_test = X.loc[data_train.shape[0]:]

In [240]:
y_train = data_train.Price
y_train.head()

0    11912523.48
1     7993374.48
2     5112900.00
3    22563005.40
4    16037611.20
Name: Price, dtype: float64

In [241]:
y_train.to_csv('../data/transform/y_train.csv',index=False)
X_train.to_csv('../data/clean/X_train_clean.csv',index=False)
X_test.to_csv('../data/clean/X_test_clean.csv',index=False)
X.to_csv('../data/clean/X_clean.csv',index=False)

In [242]:
pd.DataFrame(outliers_index).to_csv('../data/transform/outliers_index.csv',index=False)